In [1]:
import numpy as np
import pandas as pd
import requests
import time
import logging
from datetime import datetime
from citipy import citipy
from config import weather_api_key

In [2]:
size = 2000
lats = np.random.uniform(low=-90.000, high=90.000, size=size)
lngs = np.random.uniform(low=-180.000, high=180.000, size=size)

df_coordinates = pd.DataFrame({'Lat':lats, 'Lng':lngs})
df_coordinates.head()

,Lat,Lng
0,-79.294485,45.222353
1,20.294310,-132.447524
2,-80.809728,-56.713456
3,-52.277640,107.492854
4,-52.540883,34.194152


In [3]:
cities = df_coordinates.apply(lambda x: citipy.nearest_city(x['Lat'], x['Lng']).city_name, axis=1)

In [4]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [5]:
# Create an empty list to hold the weather data.
city_data = []

logging.basicConfig(filename='weather_data.log', encoding='utf-8', level=logging.INFO)
# Print the beginning of the logging.
logging.info("Beginning Data Retrieval     ")
logging.info("-----------------------------")

# Create counters.
#record_count = 1
#set_count = 1

for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes.
#     if (i % 50 == 0 and i >= 50):
#         set_count += 1
#         record_count = 1
#         time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    logging.info(f"Processing Record {i + 1} | {city}")
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description,
                          "Date": city_date})        

# If an error is experienced, skip the city.
    except:
        logging.warning("City not found. Skipping...")
        pass
    
    # pause 1 seconds per record so it won't bypass the limit
    time.sleep(1)

# Indicate that Data Loading is complete.
logging.info("-----------------------------")
logging.info("Data Retrieval Complete      ")
logging.info("-----------------------------")


In [6]:
city_data_df = pd.DataFrame(city_data)

In [10]:
city_data_df.to_csv('WeatherPy_Database.csv', index=False)

In [11]:
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,Port Alfred,-33.5906,26.8910,62.82,83,99,7.45,ZA,overcast clouds,2022-08-11 19:47:09
1,Lompoc,34.6391,-120.4579,78.94,74,40,14.97,US,mist,2022-08-11 19:47:11
2,Ushuaia,-54.8000,-68.3000,38.82,87,40,2.30,AR,scattered clouds,2022-08-11 19:43:07
3,Albany,42.6001,-73.9662,84.79,51,93,10.74,US,overcast clouds,2022-08-11 19:44:20
4,Port Alfred,-33.5906,26.8910,62.82,83,99,7.45,ZA,overcast clouds,2022-08-11 19:47:09
